In [2]:
import time
import random
import os
import pandas as pd
import csv
import numpy as np
from itertools import chain
from pdp_lib import processing_google_map as proc
from pdp_lib import util
from GA_lib import GA_multi_depot as GA
from GA_lib import operation_multi_depot as operation
from GA_lib import evaluate_multi_depot as evaluate
from statistics import mode

In [75]:
def solve_and_print(filepath, rounds = 10, population_size = 100, generations=2000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    # DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    DEPOT_NUMBERS = proc.KNN_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    print(REQ_BY_DEPOTS)
    print('----------------Instance name: ' + str(filepath) + '-----------------------------------')
    print(" processing time --- %s seconds ---" % (time.time() - process_time))

    total_results = []
    for round in range(rounds):
        results_all_depots = []
        distance_all_depots = []
        time_all_depots = []
        for id_depot in range(len(DEPOTS)):
            THIS_DEP_REQS = REQ_BY_DEPOTS[id_depot]
            NODES_OF_THIS_DEPOT = proc.locations_of_this_depot(id_depot, REQ_BY_DEPOTS, LOCATIONS)
            if(len(NODES_OF_THIS_DEPOT)==0):
                results_all_depots.append([])
                distance_all_depots.append(0)
                time_all_depots.append(0)
                continue
            # ############### SOLVING THE PROBLEMS !!!!!!!! ######################################
            pops_create_time = time.time()
            calculation_time = time.time()
            print('#################### Round :' +str(round+1)+' ############################')
            ## Initialize the populations
            populations = []
            for _ in range(population_size):
                chromosome = GA.initialize_Feasible_chromosome(DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS, LoadCapacities)
                populations.append(chromosome)
            # print("Populations creation time --- %s seconds ---" % (time.time() - pops_create_time))
            ## Crossovers and mutate ##
            GA_time = time.time()

            ## Evaluate fitnesses of all populations ##
            best_fitness_so_far = -99999999
            for gen in range(generations):
                fitness_table = []
                for chromosome in populations:
                    fitness_table.append(
                        evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,
                                                   id_depot))
                populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
                populations.pop()
                populations.pop()
                elite1 = populations.pop(0)
                elite2 = populations.pop(0)


                ## Tournament Selection ##
                # id1, id2 = random.randrange(0, len(populations)), random.randrange(0, len(populations))
                id1,id2,id3,id4 = random.sample(range(0, len(populations)), 4)
                fitness1,fitness2,fitness3,fitness4 = fitness_table[id1],fitness_table[id2],fitness_table[id3],fitness_table[id4]
                candidate1,candidate2,candidate3,candidate4 = populations[id1], populations[id2],populations[id3],populations[id4]
                parent1 = operation.tournament_selection(candidate1,candidate2,fitness1,fitness2)
                parent2 = operation.tournament_selection(candidate3,candidate4,fitness3,fitness4)

                ## Crossing over !! ####
                child1, child2 = operation.crossover(DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, parent1, parent2,
                                                     DEMANDS, LoadCapacities, DISTANCES_FROM_DEPOTS,
                                                     DISTANCES_TO_DEPOTS, id_depot, crossoverRate)
                # if(not evaluate.haveEqualNodes(child1,child2,LOCATIONS)):
                #         print('note have Equal nodes, Crossover Bug!!!!!'+ str(gen))
                #         break
                #
                ## Mutation ##
                child1 = operation.mutate(child1, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                child2 = operation.mutate(child2, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                if (not evaluate.haveEqualNodes(child1, child2, NODES_OF_THIS_DEPOT)):
                    print('note have Equal nodes, Mutation Bug!!!!!' + str(gen))
                    break


                ## Evaluate Fitness of the best ones ###
                current_fitness = evaluate.chromosomeFitness(child1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                             DISTANCES_TO_DEPOTS, id_depot)
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(child2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))

                ## Add them back into populations ###
                populations.append(child1)
                populations.append(child2)
                populations.append(elite1)
                populations.append(elite2)

                ### Check break conditions ####
                if (current_fitness > best_fitness_so_far):
                    best_fitness_so_far = current_fitness
                    bestFitGen = gen
                    # print('#### New Best Fitness !! , Best so far is :' + str(10000.0 / best_fitness_so_far) + '#####')
                    # print('#### This Generation: ' + str(gen) + '#######')
                if (gen - bestFitGen >= 1000):
                    # print('#### Break Generation: ' + str(gen) + '#######')
                    break
                # print('############# Round-'+str(round+1) +',DEPOT-'+str(id_depot)+','+ 'Generation:' + str(gen + 1) + ' #########################')
                # print('This Gen Distance :' + str(10000.0 / current_fitness))

            ### Evaluate results of current DEPOT ####
            fitness_table = []
            for chromosome in populations:
                fitness_table.append(
                    evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot))
            populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
            # print("GA time --- %s seconds ---" % (time.time() - GA_time))
            best_chromosome = populations[0]
            computational_time = time.time() - calculation_time
            # print("Total Calculation time --- %s seconds ---" % (computational_time))
            distance = evaluate.chromosomeRoutesDistance(best_chromosome,DISTANCES,DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,id_depot)
            # print('Distances of the best chromosome: ' + str(distance))
            # print(best_chromosome)

            ## Memo the results ##
            results_all_depots.append(best_chromosome)
            distance_all_depots.append(distance)
            time_all_depots.append(computational_time)
        total_results.append((results_all_depots,distance_all_depots,time_all_depots))
        print(results_all_depots)
    return total_results

In [4]:
def mean_results(results):
    solutions = []
    cal_times = []
    distances = []
    for round, round_result in enumerate(results):
        sol = results[round][0]
        dist = results[round][1]
        ct = results[round][2]

        solutions.append(sol)
        distances.append(dist)
        cal_times.append(ct)
    distances = np.array(distances)
    distances = pd.DataFrame(distances)
    distances = np.mean(distances, axis=0)
    cal_times = np.array(cal_times)
    cal_times = pd.DataFrame(cal_times)
    cal_times = np.mean(cal_times, axis=0)
    return solutions,distances, cal_times


In [5]:
def create_worst_case(filepath, rounds = 1, population_size = 100, generations=3000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    # DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    DEPOT_NUMBERS = proc.KNN_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    
    res = GA.initialize_WorstCase_Chromosome(REQUESTS)
    
    
    dist_worst = 0
    for [idx,_,[p,d]] in res:
        depot = int(DEPOT_NUMBERS[idx]) 
        dist_worst += (DISTANCES_FROM_DEPOTS[depot][p]+DISTANCES[p][d]+DISTANCES_TO_DEPOTS[d][depot])

    return res,dist_worst

# Automated run

In [10]:
path = 'pdp_instances/GOO/doing/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    results = solve_and_print(filepath)
    solutions,distances,cal_times = mean_results(results)
    all_results.append((solutions,distances,cal_times))

----------------Instance name: pdp_instances/GOO/doing/goo9.csv-----------------------------------
 processing time --- 0.3500196933746338 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


...

#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


[[[3, [29], [62, 80]], [2, [26, 11], [29, 57, 59, 47]], [1, [20, 13, 8], [44, 33, 32, 30, 22, 49]], [0, [21, 12, 16, 19, 15, 14], [38, 43, 45, 39, 37, 34, 31, 36, 11, 18, 14, 9]]], [[1, [39, 28, 24], [55, 61, 79, 26, 7, 53]], [0, [22], [48, 5]]], [[2, [32, 2, 9, 10, 3, 30, 1, 0], [8, 4, 64, 63, 13, 25, 23, 16, 3, 1, 50, 66, 2, 41, 35, 46]], [1, [36, 35, 31, 7, 5], [72, 70, 65, 6, 12, 21, 28, 24, 20, 17]]], [[0, [34, 27], [69, 60, 58, 77]]], [[3, [37, 17, 23], [76, 51, 40, 52, 74, 75]], [2, [4], [10, 54]], [0, [38, 33, 6, 18, 25], [56, 78, 42, 27, 19, 15, 68, 67, 71, 73]], [1, [], []]]]


# The GOO Instances

# Average list by columns

In [5]:
# filename = 'pdp_instances/GOO/goo1.csv'

In [6]:
# filepath = filename 
# filename = filename.rsplit('/', 1)[1]
# filename = filename.split('.')[0]
# print( 'Instance name : '+str(filename))


Instance name : goo1


In [7]:
# results = solve_and_print(filepath)

----------------Instance name: pdp_instances/GOO/goo1.csv-----------------------------------
 processing time --- 1.0620617866516113 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


In [8]:
# results

[([[[2, [15], [35, 49]],
    [1, [19, 16], [41, 37, 34, 22]],
    [0, [13, 4, 6], [15, 31, 43, 18, 48, 42]]],
   [[1, [38], [79, 61]], [0, [39, 25, 24], [52, 51, 80, 21, 5, 10]]],
   [[6, [5, 3, 8], [9, 24, 16, 12, 11, 2]],
    [1, [21, 2, 12], [7, 45, 33, 30, 23, 25]],
    [5, [], []],
    [2, [32, 27, 11, 10, 33], [29, 63, 55, 77, 69, 28, 27, 64, 71, 65]],
    [4, [], []],
    [0, [31, 23, 7, 18], [62, 47, 40, 19, 20, 13, 50, 53]],
    [3, [9, 14, 0, 34], [32, 26, 3, 4, 67, 68, 66, 14]]],
   [[0, [30], [60, 58]]],
   [[3, [], []],
    [0, [17, 26, 22, 20], [54, 46, 44, 36, 39, 70, 78, 72]],
    [1, [29, 1, 28], [56, 57, 38, 1, 6, 59]],
    [2, [37, 35, 36], [74, 75, 73, 76, 17, 8]]]],
  [62342.0, 32954.0, 156140.0, 11241.0, 114938.0],
  [5.082298040390015,
   3.2432212829589844,
   23.598564386367798,
   1.1332643032073975,
   6.455474376678467])]

In [9]:
# res =results[0]
# SOL,DIST,CT = res[0],res[1],res[2]


In [11]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [65]:
worst_sol,distance_worst = create_worst_case('pdp_instances/GOO/goo9.csv')
distance_worst

733752.0

In [66]:
# SOL[2]
count_num_vehicle([worst_sol])

[40]

In [67]:

worst_sol

[[0, [0], [1, 2]],
 [1, [1], [3, 35]],
 [2, [2], [4, 64]],
 [3, [3], [8, 13]],
 [4, [4], [10, 54]],
 [5, [5], [12, 24]],
 [6, [6], [19, 71]],
 [7, [7], [20, 17]],
 [8, [8], [22, 49]],
 [9, [9], [23, 41]],
 [10, [10], [25, 16]],
 [11, [11], [29, 59]],
 [12, [12], [31, 18]],
 [13, [13], [33, 30]],
 [14, [14], [36, 9]],
 [15, [15], [38, 11]],
 [16, [16], [39, 14]],
 [17, [17], [40, 74]],
 [18, [18], [42, 67]],
 [19, [19], [43, 37]],
 [20, [20], [44, 32]],
 [21, [21], [45, 34]],
 [22, [22], [48, 5]],
 [23, [23], [51, 75]],
 [24, [24], [55, 26]],
 [25, [25], [56, 27]],
 [26, [26], [57, 47]],
 [27, [27], [60, 77]],
 [28, [28], [61, 7]],
 [29, [29], [62, 80]],
 [30, [30], [63, 50]],
 [31, [31], [65, 21]],
 [32, [32], [66, 46]],
 [33, [33], [68, 73]],
 [34, [34], [69, 58]],
 [35, [35], [70, 6]],
 [36, [36], [72, 28]],
 [37, [37], [76, 52]],
 [38, [38], [78, 15]],
 [39, [39], [79, 53]]]

In [10]:
# results = [([[[2,[16,19],[41,37,34,22]],[1,[13,4,6],[15,31,43,18,48,42]],[0,[15],[35,49]]],[[1,[39,25,24],[52,51,80,21,5,10]],[0,[38],[79,61]]],[[2,[23,7,21],[19,47,45,50,53,25]],[5,[12,31,18],[62,40,30,23,20,13]],[3,[2,32,14,11,27,10,33],[32,7,4,29,33,63,55,77,69,28,27,64,71,65]],[1,[],[]],[0,[34,9,8,0],[3,26,24,68,67,66,2,14]],[4,[3,5],[9,16,12,11]]],[[0,[30],[60,58]]],[[3,[37,35,36],[74,75,73,76,17,8]],[2,[],[]],[1,[29,28,1],[56,57,38,1,6,59]],
                                                                                                                                                                                                                                                                                                                                                                                       [0,[26,22,20,17],[54,46,44,36,39,70,78,72]]]])]

In [16]:
# res = results[0]

In [18]:
# res[0]

[[2, [16, 19], [41, 37, 34, 22]],
 [1, [13, 4, 6], [15, 31, 43, 18, 48, 42]],
 [0, [15], [35, 49]]]

# Writing to CSV

In [23]:
solutions,distances,cal_times = mean_results(all_results)
dist = [dist for _,dist,_ in all_results]
dist = np.array(dist)
df = pd.DataFrame(dist)
df.to_csv('distances_goo11.csv')
CT = [ct for _,_,ct in all_results]
CT = np.array(CT)
df_CT = pd.DataFrame(CT)
df_CT.to_csv('CT_goo11.csv')

# Average list by columns

In [8]:
def avg_table(table):
    transposed = zip(*table)
    avg = lambda items: float(sum(items)) / len(items)
    averages = map(avg, transposed)
    averages = list(averages)
    return  averages

In [9]:
# for sol in solutions:
num_vehicles  = []
i = 0
for res in all_results:
    solutions = res[0]
    arr_temp = []
    for sol in solutions:
        num_temp = count_num_vehicle(sol)
        arr_temp.append(num_temp)
    avg_temp = avg_table(arr_temp)
    print(avg_temp)
    num_vehicles.append(avg_temp)

[3.7, 1.0, 3.0, 0.0, 2.0]


In [10]:

num_vehicles = np.array(num_vehicles)
df_num_vehicles = pd.DataFrame(num_vehicles)
df_num_vehicles.to_csv('num_veh_goo.csv')

([[[3, [39, 28], [80, 61, 26, 44]],
   [2, [29, 11, 16, 21], [62, 52, 40, 29, 38, 35, 27, 31]],
   [1, [35, 37, 34, 38], [76, 75, 73, 71, 34, 30, 42, 41]],
   [0, [32, 33, 36, 22], [65, 66, 74, 53, 79, 46, 43, 45]]],
  [[0, [17, 15], [39, 47, 11, 19]]],
  [[3, [31, 30, 25], [63, 64, 70, 56, 12, 2]],
   [2, [9, 3, 1], [25, 23, 9, 4, 16, 7]],
   [0,
    [12, 0, 18, 6, 2, 8, 10],
    [3, 28, 48, 20, 13, 5, 6, 1, 8, 24, 33, 32, 22, 15]]],
  [],
  [[2,
    [27, 20, 14, 7, 23, 4],
    [54, 37, 51, 77, 59, 60, 68, 21, 17, 14, 50, 57]],
   [1, [], []],
   [0, [26, 19, 5, 13, 24], [58, 36, 10, 18, 49, 55, 69, 72, 78, 67]]]],
 [138762.0, 28910.0, 76408.0, 0, 128962.0],
 [16.508774280548096,
  1.6227107048034668,
  15.153313875198364,
  0,
  10.725628137588501])

In [30]:
temp = all_results[0][0][0]

In [36]:
temp = temp[2]

In [44]:
a = set(temp[0][2])

In [45]:
b = set(temp[1][2])

In [49]:
c = list(a.union(b))

In [59]:
c =set(c)

In [55]:
temp = [{8: (22, 49), 11: (29, 59), 12: (31, 18), 13: (33, 30), 14: (36, 9), 15: (38, 11), 16: (39, 14), 19: (43, 37), 20: (44, 32), 21: (45, 34), 26: (57, 47), 29: (62, 80)}, {22: (48, 5), 24: (55, 26), 28: (61, 7), 39: (79, 53)}, {0: (1, 2), 1: (3, 35), 2: (4, 64), 3: (8, 13), 5: (12, 24), 7: (20, 17), 9: (23, 41), 10: (25, 16), 30: (63, 50), 31: (65, 21), 32: (66, 46), 35: (70, 6), 36: (72, 28)}, {27: (60, 77), 34: (69, 58)}, {4: (10, 54), 6: (19, 71), 17: (40, 74), 18: (42, 67), 23: (51, 75), 25: (56, 27), 33: (68, 73), 37: (76, 52), 38: (78, 15)}]

In [72]:
temp[2]
s = set([x for x in temp[2].values()])
s

{(1, 2),
 (3, 35),
 (4, 64),
 (8, 13),
 (12, 24),
 (20, 17),
 (23, 41),
 (25, 16),
 (63, 50),
 (65, 21),
 (66, 46),
 (70, 6),
 (72, 28)}

In [73]:
worst_sol2 = [ x for x in worst_sol if tuple(x[2]) in s]

In [74]:
worst_sol2

[[0, [0], [1, 2]],
 [1, [1], [3, 35]],
 [2, [2], [4, 64]],
 [3, [3], [8, 13]],
 [5, [5], [12, 24]],
 [7, [7], [20, 17]],
 [9, [9], [23, 41]],
 [10, [10], [25, 16]],
 [30, [30], [63, 50]],
 [31, [31], [65, 21]],
 [32, [32], [66, 46]],
 [35, [35], [70, 6]],
 [36, [36], [72, 28]]]